# Pipeline test notebook

### Import requirements

In [3]:
%matplotlib ipympl

import glob
import cv2
import numpy
import matplotlib.pyplot as plt

from ImageProcessing import ImageProcessing
from StereoMatcher import StereoMatcher

### Loading images

In [4]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

### Select image pair

In [5]:
imageNumber = 1
imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

### View images

In [6]:
cv2.imshow("left/right", numpy.hstack([imageL, imageR]))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Load calibrations and other data

In [7]:
imageProcessing = ImageProcessing()
imageProcessing.loadMonoCalibrationResults()
imageProcessing.loadStereoCalibration()
imageProcessing.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [8]:
stereoMatcher = StereoMatcher("SGBM", \
                vertical=True, createRightMatcher=False)
stereoMatcher.referenceDispToDepthMatrix(\
imageProcessing.dispToDepthMatrix)

imageProcessing.initUndistortRectifyMap()

Reading from data/parametersSGBM.json


### Convert to grayscale and undistort

In [9]:
imageProcessing.convertToGrayscale(imageL, imageR)
imageProcessing.undistortRectifyRemap(imageProcessing.grayImageL, \
                                        imageProcessing.grayImageR)

### View results of above

In [10]:
cv2.imshow("left/right undistort", numpy.hstack(\
    [imageProcessing.undistortImageL, imageProcessing.undistortImageL]))

cv2.imshow("horizontal epipolar", \
    imageProcessing.drawHorEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))

cv2.imshow("vertical epipolar", \
    imageProcessing.drawVertEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))
        
cv2.waitKey(0)
cv2.destroyAllWindows()

### Compute disparity map

In [11]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessing.undistortImageL, \
                grayImageR=imageProcessing.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()

### View disparity map

In [12]:
cv2.imshow("disparity", stereoMatcher.disparityMapL)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Compute point cloud

In [13]:
points = cv2.reprojectImageTo3D(stereoMatcher.disparityMapL, \
            stereoMatcher.dispToDepthMatrix)
pointCloud = points.reshape((points.shape[0]*points.shape[1],3))[0::20]/100000
pointCloud = pointCloud[pointCloud[:, 2]>0]
print (points.shape)
print (pointCloud.shape)

(360, 640, 3)
(10158, 3)


### View point cloud

In [14]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = "3d")
ax.scatter(pointCloud[:,0], pointCloud[:,1], pointCloud[:,2], s=2)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …